In [ ]:
using BenchmarkTools
using Combinatorics

include("../phase1/node.jl")
include("../phase1/edges.jl")
include("../phase1/graph.jl")
include("../phase1/read_stsp.jl")
include("../phase2/comp_connexes.jl")
include("../phase1/main.jl")
include("../phase2/queue.jl")
include("../phase2/heuristics.jl")
include("../phase2/Kruskal.jl")
include("../phase2/prims_algorithm.jl")

In [ ]:
tree_root = TreeNode("1", 0)
child1 = TreeNode("2", 0)
child2 = TreeNode("3", 0)
child3 = TreeNode("4", 0)
child4 = TreeNode("5",0)
child5 = TreeNode("6",0)
child6 = TreeNode("7",0)

tree = Tree("test tree", Vector{TreeNode{Int}}())

first_row = [ child1, child2, child3]
second_row = [child4, child6]
add_node!(tree, tree_root)
for child in first_row
    add_node!(tree, child, parent = tree_root)
end
for child in second_row
    add_node!(tree, child, parent = child3)
end
add_node!(tree, child5, parent = child4)
show(tree)

In [ ]:
find_root(tree)

In [ ]:
children_loc(child4)

In [ ]:
println(tree_root)

In [ ]:
n1= TreeNode("1", 0)
n2 = TreeNode("2", 0)
n3 = TreeNode("3", 0)
n4= TreeNode("4", 0)
t1 = Tree("test tree" , Vector{TreeNode{Int}}())
t1 = add_node!(t1, n1)
t1 = add_node!(t1, n2, parent = n1)
t1 = add_node!(t1, n3)
t1 = add_node!(t1, n4, parent = n3)
rank_union!(t1, n1, n3)


In [ ]:
show(t1)

In [ ]:
"""crée une liste d'adjacence a partir d'un graphe"""
function adjacency_dict( graph::Graph) 
    #Dictionaire de adjacence
    adj_dict = Dict()
    #Dictionaire de correspondance entre les noeuds et les items de la file de priorite
    correspondance_dict = Dict()
    #Chaque noeud est une clef du dictionaire de adjacence
    for (i, node) in enumerate(nodes(graph))
        adj_dict[i] = Dict()
        correspondance_dict[name(node)] = i
    end
    #Ajoute les voisins de chaque noeud dans le dictionaire de adjacence
    for edge in edges(graph)
        node1, node2 = nodes(edge)
        idx1 = correspondance_dict[name(node1)]
        idx2 = correspondance_dict[name(node2)]
        adj_dict[idx1][idx2] = weight(edge)
        adj_dict[idx2][idx1] = weight(edge)
    end
    return adj_dict
end

"""Crée une file de priorite des arbres à partir d'un graphe. Le noeud de depart a une priorite de 0 et les autres ont une priorite de Inf"""
function prims_priority_queue(graph::Graph{Y,T}, start_node_name::String) where {Y,T}
    priority_queue = PriorityQueue{PriorityItem{TreeNode{Y}}}()
    tree = Tree(name(graph), Vector{TreeNode{Y}}())
    for node in nodes(graph)
        if name(node) == start_node_name
            blank_node = TreeNode(name(node), data(node))
            priority_item = PriorityItem( 0 , blank_node)
        else
            blank_node = TreeNode(name(node), data(node))
            priority_item = PriorityItem( Inf, blank_node)
        end
        add_node!(tree, blank_node)
        push!(priority_queue, priority_item)
    end
    return priority_queue, tree
end

"""Implementation de l'algorithme de Prim"""
function prims_algorithm(graph::Graph{Y,T}; start_node_name::Any = nothing) where {Y,T}
    #initialisation
    if isnothing(start_node_name)
        start_node_name = name(nodes(graph)[1])
    end
    #initialisation de la file de priorite et du dictionaire d'adjacence
    priority_queue, tree = prims_priority_queue(graph, start_node_name)
    adjacency_list = adjacency_dict(graph)
    #sauvegarde de la racine
    root = poplast!(priority_queue)
    priority_node = root
    #boucle principale
    while !is_empty(priority_queue)
        for  (neighbor_idx, edge_weight) in adjacency_list[index(data(priority_node))]
            #Met a jour la priorite du voisin si elle est dans le priority_que et elle est plus petite que la priorite actuelle
            for item in priority_queue.items
                #println("nodes(tree)[neighbor_idx] = ", nodes(tree)[neighbor_idx])
                if index(data(item)) == neighbor_idx
                    if  edge_weight < priority(item)
                        change_parent!(tree, nodes(tree)[neighbor_idx], data(priority_node))
                        change_rank!( nodes(tree)[neighbor_idx], edge_weight)
                        priority!(item, edge_weight)
                    end
                break
                end
            end
        end
        #prend le noeud avec la plus petite priorite
        priority_node = poplast!(priority_queue)         
    end
    return tree, data(root)
end

In [ ]:
graphe_test = Graph("Test",Node{Vector{Float64}}[],Edge{Int,Vector{Float64}}[])

#Nodes 
nodea = Node("a",[0.])
nodeb = Node("b",[0.])
nodec = Node("c",[0.])
noded = Node("d",[0.])
nodee = Node("e",[0.])
nodef = Node("f",[0.])
nodeg = Node("g",[0.])
nodeh = Node("h",[0.])
nodei = Node("i",[0.])
node_list = [nodea,nodeb,nodec, noded,nodee, nodef, 
                nodeg, nodeh, nodei]

#Edges
edge1 = Edge(nodea,nodeb, 4)
edge2 = Edge(nodea,nodeh, 8)
edge3 = Edge(nodeb, nodeh, 11)
edge4 = Edge(nodeb, nodec, 8)
edge5 = Edge(nodeh, nodei, 7)
edge6 = Edge(nodeh, nodeg, 1)
edge7 = Edge(nodeg, nodei, 6)
edge8 = Edge(nodeg, nodef, 2)
edge9 = Edge(nodec, nodef, 4)
edge10 = Edge(nodei, nodec, 2)
edge11 = Edge(nodef, nodee, 10)
edge12 = Edge(nodee, noded, 9)
edge13 = Edge(nodec,noded, 7)
edge14 = Edge(noded, nodef, 14)
edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
                edge7, edge8, edge9, edge10, edge11,
                edge12, edge13, edge14]

##Construction du graphe
for i in node_list
    add_node!(graphe_test,i)
end
for i in edge_list
    add_edge!(graphe_test, i)
end


In [ ]:
priority_queue, tree = prims_priority_queue(graphe_test, "a")

#@benchmark p_tree, root = prims_algorithm(graphe_test, start_node_name = "a")

In [ ]:
#@benchmark kruskal(graphe_test)

In [ ]:
@time p_tree1, root1 = prims_algorithm(graphe_test, start_node_name = "a")

In [ ]:
swiss_42_graph, swiss_42_nodes = graph_from_tsp("../../instances/stsp/swiss42.tsp","graphe1")
println("running Prim's algorithm on swiss42")
@time swiss_42_tree, root2 = prims_algorithm(swiss_42_graph)
swiss_42_graph = tree_to_graph(swiss_42_tree, root2)
sum_of_weights(swiss_42_graph)

In [ ]:
graphe1, graphe1_nodes = graph_from_tsp("../../instances/stsp/bays29.tsp","graphe1")
println("running Prim's algorithm")
@time bays_29_tree, root3 = prims_algorithm(graphe1)
bays_29_graph = tree_to_graph(bays_29_tree, root3)

sum_of_weights(bays_29_graph)

## Algorithm de Held et Karp 

In [ ]:
# nodea = Node("a",[0.])
# nodeb = Node("b",[0.])
# nodec = Node("c",[0.])
# noded = Node("d",[0.])
# nodee = Node("e",[0.])
# node_list = [nodea,nodeb,nodec, noded,nodee]
# #Edges are fully connected
# edge1 = Edge(nodea,nodeb, 4.)
# edge2 = Edge(nodea,nodec, 8.)
# edge3 = Edge(nodea, noded, 11.)
# edge4 = Edge(nodea, nodee, 8.)
# edge5 = Edge(nodeb, nodec, 7.)
# edge6 = Edge(nodeb, noded, 1.)
# edge7 = Edge(nodeb, nodee, 6.)
# edge8 = Edge(nodec, noded, 2.)
# edge9 = Edge(nodec, nodee, 4.)
# edge10 = Edge(noded, nodee, 7.)
# edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
#                 edge7, edge8, edge9, edge10]

# tsp_test = Graph("Test",node_list,edge_list)
# tsp_test

nodea = Node("a",[0.])
nodeb = Node("b",[0.])
nodec = Node("c",[0.])
noded = Node("d",[0.])
nodee = Node("e",[0.])
node_list = [nodea,nodeb,nodec, noded,nodee]
#Edges are fully connected
edge1 = Edge(nodea,nodeb, 3.)
edge2 = Edge(nodea,nodec, 2.)
edge3 = Edge(nodea, noded, 11.)
edge4 = Edge(nodea, nodee, 9.)
edge5 = Edge(nodeb, nodec, 7.)
edge6 = Edge(nodeb, noded, 12.)
edge7 = Edge(nodeb, nodee, 6.)
edge8 = Edge(nodec, noded, 6.)
edge9 = Edge(nodec, nodee, 4.)
edge10 = Edge(noded, nodee, 7.)
edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
                edge7, edge8, edge9, edge10]

tsp_test = Graph("Test",node_list,edge_list)
tsp_test



In [ ]:
#Fully connected graph with 7 edges
a = Node("a",[0.])
b = Node("b",[0.])
c = Node("c",[0.])
d = Node("d",[0.])
e = Node("e",[0.])
f = Node("f",[0.])
g = Node("g",[0.])
node_list = [nodea,nodeb,nodec, noded,nodee, nodef, nodeg]
#Edges are fully connected
e1 = Edge(nodea,nodeb, 4.)
edge2 = Edge(nodea,nodec, 8.)
edge3 = Edge(nodea, noded, 11.)
edge4 = Edge(nodea, nodee, 8.)
edge5 = Edge(nodea, nodef, 7.)
edge6 = Edge(nodea, nodeg, 1.)
edge7 = Edge(nodeb, nodec, 6.)
edge8 = Edge(nodeb, noded, 2.)
edge9 = Edge(nodeb, nodee, 4.)
edge10 = Edge(nodeb, nodef, 7.)
edge11 = Edge(nodeb, nodeg, 2.)
edge12 = Edge(nodec, noded, 7.)
edge13 = Edge(nodec, nodee, 1.)
edge14 = Edge(nodec, nodef, 6.)
edge15 = Edge(nodec, nodeg, 3.)
edge16 = Edge(noded, nodee, 5.)
edge17 = Edge(noded, nodef, 4.)
edge18 = Edge(noded, nodeg, 8.)
edge19 = Edge(nodee, nodef, 2.)
edge20 = Edge(nodee, nodeg, 7.)
edge21 = Edge(nodef, nodeg, 3.)
edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
                edge7, edge8, edge9, edge10, edge11,
                edge12, edge13, edge14, edge15, edge16,
                edge17, edge18, edge19, edge20, edge21]
#creates the graph
tsp_test2 = Graph("Test2",node_list,edge_list)

In [ ]:
function h_k_exact_algorithm(g::Graph ; start_node_name::Any = nothing)
    #if no start node name, first node is start node
    if isnothing(start_node_name)
        start_node_name = name(nodes(g)[1])
    end
    #move start node to first position of graph
    for (i, node) in enumerate(nodes(g))
        if name(node) == start_node_name
            g.nodes[1], g.nodes[i] = g.nodes[i], g.nodes[1]
            break
        end
    end
    #create adjacency list
    adjacency_list = adjacency_dict(g)
    distance_dict = Dict()
    for k in 2:length(nodes(g))
        distance_dict[(Set([k]), k)] = Dict("c" =>adjacency_list[1][k], "p"=> [1, k])
    end
    for s in 2:length(nodes(g))-1
        for subset in combinations(2:length(nodes(g)), s)
            for k in subset
                min_dist = Inf
                for m in subset 
                    if m !=k
                        predecessor = distance_dict[(setdiff(Set(subset), Set([k])), m)]
                        #println("predecessor", predecessor)
                        dist = predecessor["c"] + adjacency_list[m][k]
                        if dist < min_dist
                            min_dist = dist
                            min_m = m
                            new_list = copy(predecessor["p"])
                            append!(new_list, k)
                            distance_dict[(Set(subset), k)] = Dict("c" => min_dist, "p" => new_list)
                        end
                    end
                end
            end
        end
    end

    #returns the minimum path from the start node that goes to all of the other paths
    min = Inf
    final_path = []
    for k in 2:length(nodes(g))
        distance = distance_dict[(Set(2:length(nodes(g))), k)]["c"] + adjacency_list[k][1]
        println("distance = ", distance)
        if distance < min
            println("distance = ", distance)
            min = distance
            final_path = copy(distance_dict[(Set(2:length(nodes(g))), k)]["p"])
            append!(final_path, 1)
        end
    end
    return min, final_path
end


h_k_algorithm(tsp_test)


In [ ]:


function get_shortest_edges(graph::Graph, departure_node::Node)
    shortest_edge_vec = Vector{Edge}(undef, 2)
    shortest_dist_vec = Vector{Float64}([Inf, Inf])
    for  edge in edges(graph)
        node1, node2 = nodes(edge)
        if name(node1) == name(departure_node) || name(node2) == name(departure_node)
            #copies over the previous best to second best
            if weight(edge) < shortest_dist_vec[1]
                if shortest_dist_vec[1]< shortest_dist_vec[2]
                    shortest_dist_vec[2] = shortest_dist_vec[1]
                    shortest_edge_vec[2] = shortest_edge_vec[1]
                end
                shortest_dist_vec[1] = weight(edge)
                shortest_edge_vec[1] = edge
            elseif weight(edge) < shortest_dist_vec[2]
                shortest_dist_vec[2] = weight(edge)
                shortest_edge_vec[2] = edge
            end
        end
    end
    return  shortest_edge_vec[1], shortest_edge_vec[2]
end



function find_one_tree(graph::Graph, departure_node::Node, adjacency_list::Dict)
    #Copies graph to feed into prim's algorithm
    start_graph = deepcopy(graph)
    #removes the departure node from the graph and saves its index
    idx = remove_node!(start_graph, departure_node)
    remove_edges!(start_graph, departure_node)
    #uses prims algorithm to find the one_tree
    one_tree, root = prims_algorithm(start_graph, start_node_name = name(departure_node))
    #converts the prims algorithm to a graph
    one_tree_graph = tree_to_graph(one_tree, root)
    #finds the second shortest edge from the departure node
    shortest_edge_1, shortest_edge_2   = get_shortest_edges(graph, departure_node)
    add_node!(one_tree_graph, departure_node, idx)
    add_edge!(one_tree_graph,shortest_edge_1)
    add_edge!(one_tree_graph,shortest_edge_2)
    one_tree_distance = sum_of_weights(one_tree_graph)
    return one_tree_distance, one_tree_graph
end

#Returns the degrees of nodes in  a graph
function degree(graph::Graph)
    adjacency_list = adjacency_dict(graph)
    degrees = []
    for i in 1:length(nodes(graph))
        push!(degrees, length(keys(adjacency_list[i])))
    end
    return degrees
end

function update_edge_weights!(graph::Graph, pis::Vector{Float64})
    correspondance_dict = Dict()
    for (i, node) in enumerate(nodes(graph))
        correspondance_dict[name(node)] = i
    end
    for edge in edges(graph)
        node1, node2 = nodes(edge)
        node1_idx = correspondance_dict[name(node1)]
        node2_idx = correspondance_dict[name(node2)]
        new_weight = weight(edge) + pis[node1_idx] + pis[node2_idx]
        set_weight!(edge, new_weight)
    end
end

"""subgradient heuristic for calculating a minimal tour"""
function lkh_subgradient(start_graph::Graph;  departure_node::Union{Node, Nothing} = nothing)
    start_weight = sum_of_weights(start_graph)
    no_nodes = length(nodes(start_graph))
    if isnothing(departure_node)
        departure_node = nodes(start_graph)[1]
    end
    graph = deepcopy(start_graph)
    k = 0
    w = -Inf
    pis = zeros(length(nodes(graph)))
    adjacency_list = adjacency_dict(graph)
    while true && k < 10000
        total_distance, one_tree = find_one_tree(graph, departure_node, adjacency_list)
        weights_k = total_distance - 2 * sum(pis)
        w = max(w, weights_k)
        v_k = degree(one_tree) .- 2

        
        if v_k == zeros(length(nodes(graph)))
            return total_distance, one_tree
        end
        t_k = start_weight/(100*no_nodes)
        pis = pis + t_k * v_k
        println("k ", k, "v_k", v_k)
        println("pis", pis)
        k = k + 1
        update_edge_weights!(graph, pis)
    end
    return Inf, nothing
end

In [ ]:
#lkh_subgradient(tsp_test, departure_node = nodec)

In [ ]:
score, test2_graph = lkh_subgradient(tsp_test2)
score

In [ ]:
gr17_graph, gr17_nodes = graph_from_tsp("../../instances/stsp/gr17.tsp","graphe1")
println("running h_k_algorithm on gr17")
@time total_distance, one_tree = lkh_subgradient(gr17_graph)

In [ ]:
"""Brute force calculates TSP solution"""
function brute_force_tsp(g::Graph, start_node::Any)
    adjacency_list = adjacency_dict(g)
    g_nodes = nodes(g)
    nodes_perm = collect(permutations(collect(1:length(g_nodes))))
    min_dist = Inf
    min_perm = []
    for perm in nodes_perm
        dist = 0
        for i in 1:length(perm)-1
            dist += adjacency_list[perm[i]][perm[i+1]]
        end
        dist += adjacency_list[perm[end]][perm[1]]
        println("permutation", perm)
        println("dist = ", dist)
        if dist < min_dist
            min_dist = dist
            min_perm = perm
        end
        
    end
    return min_dist, min_perm
end
brute_force_tsp(tsp_test, "a")